## Initical Cross-Matching

Code to generate beautifull histograms for initial radius cross-matching

In [2]:
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u

import numpy as np
import matplotlib.pyplot as plt

In [3]:
# EMU catalogs
emu_catalog_path = '/data/mfonseca/survey_data/EMU_data/EMU_0102-32/EMU_0102-32_1comp.fits'
emu_patched_catalog_path = '/data/mfonseca/survey_data/EMU_data/EMU_0102-32/EMU_0102-32_1comp_withoutpatches.fits'
emuXviking_catalog_path = ''

# DES catalogs
desdr2_catalog_path = '/data/mfonseca/survey_data/DES_data/DESDR2_in_EMU_0102-32_magautodered.fits'
desy6gold_catalog_path = '/data/mfonseca/survey_data/DES_data/DESY6GOLD_in_EMU_0102-32_magauto.fits'

# VIKING DR5 catalogs
vikingdr5_catalog_path = '/data/mfonseca/survey_data/VIKING_data/VIKINGDR5_in_EMU_0102-32_allapermag3.fits'

# CatWISE catalogs
catwise_catalog_path = '/data/mfonseca/survey_data/CATWISE_data/CATWISE2020_in_EMU_0102-32.fits'

In [ ]:
# EMU
emu_table = Table.read(emu_catalog_path)
emu_catalog_df = emu_table.to_pandas()

emu_coords = SkyCoord(ra=emu_catalog_df['ra_deg_cont'].values * u.deg,
                      dec=emu_catalog_df['dec_deg_cont'].values * u.deg)

# DES Y6 GOLD
desy6gold_table = Table.read(desy6gold_catalog_path)
desy6gold_catalog_df = desy6gold_table.to_pandas()

desy6gold_coords = SkyCoord(ra=desy6gold_catalog_df['ra'].values * u.deg,
                         dec=desy6gold_catalog_df['dec'].values * u.deg)


# VIKING DR5
viking_table = Table.read(vikingdr5_catalog_path)
viking_catalog_df = viking_table.to_pandas()

viking_coords = SkyCoord(ra=viking_catalog_df['ra'].values * u.deg,
                         dec=viking_catalog_df['dec'].values * u.deg)

# CatWISE 2020



In [1]:
def crossmatch_separation(cat1, cat2):

    idx, d2d, _ = cat1.match_to_catalog_sky(cat2)
    separation = d2d.arcsec

    return separation


## EMU x DES

In [5]:
crossmatch_separation(emu_coords, viking_coords)

array([ 1.03029574,  0.86980563,  2.92258252, ...,  1.72575684,
       25.78516492,  1.9324179 ])

## EMU x VIKING

## EMU x CatWISE

## Combined Histogram